<a href="https://colab.research.google.com/github/sowmyakovur20/machinelearning/blob/main/NewfinalNLPBonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import pickle, glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from numpy.random import randn
from matplotlib import pyplot
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


In [72]:
path = '/content/drive/MyDrive/BonusQuestionData/data/*.bin'
files = glob.glob(path)
signal = [0,0,0,0,0]
for i,f in enumerate(files):
    with open(f, 'rb') as fp:
        signal[i] = pickle.load(fp)

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
signal1 = np.array(signal[4], dtype = 'O')
signal2 = np.array(signal[3], dtype = 'O')
signal3 = np.array(signal[2], dtype = 'O')
signal4 = np.array(signal[1], dtype = 'O')
signal5 = np.array(signal[0], dtype = 'O')


In [75]:
signal1 = pd.DataFrame(signal1, columns = ['X'])
signal1['Y'] = 0

signal2 = pd.DataFrame(signal2, columns = ['X'])
signal2['Y'] = 1

signal3 = pd.DataFrame(signal3, columns = ['X'])
signal3['Y'] = 2

signal4 = pd.DataFrame(signal4, columns = ['X'])
signal4['Y'] = 3

signal5 = pd.DataFrame(signal5, columns = ['X'])
signal5['Y'] = 4

In [76]:
data = signal1.append(signal2, ignore_index = True)
data = data.append(signal3, ignore_index = True)
data = data.append(signal4, ignore_index = True)
data = data.append(signal5, ignore_index = True)

In [77]:
data

,X,Y
0,"[17.35347938537598, 18.08359146118164, 19.4127...",0
1,"[21.62890434265137, 23.19485092163086, 23.8474...",0
2,"[27.77483749389648, 30.16787528991699, 32.0842...",0
3,"[40.64637756347656, 37.54090118408203, 36.8591...",0
4,"[59.7484130859375, 61.23234176635742, 62.69600...",0
...,...,...
1053,"[1.150460720062256, 1.058937549591064, 1.09076...",4
1054,"[3.620477437973022, 3.860167503356934, 4.19080...",4
1055,"[4.077456474304199, 4.05850076675415, 4.585644...",4
1056,"[10.45901107788086, 5.725070476531982]",4


In [78]:
size = data['X'].str.len()
m= max(size)
x = []
for i in range(0,len(data)):
    i_size = len(data['X'][i])
    old = data['X'][i]
    if i_size < m:
        new = [0] * (m - i_size)
        x.append(old + new)
    else:
        x.append(old)

In [79]:
x = np.array(x)
y = data['Y'].values

In [80]:
df = pd.DataFrame(x)
df['y'] = y

In [81]:
def r(a):
    return round(a,2)

for i in range(0,477):
    df[i] = df[i].apply(r)

In [82]:
df['y']

0       0
1       0
2       0
3       0
4       0
       ..
1053    4
1054    4
1055    4
1056    4
1057    4
Name: y, Length: 1058, dtype: int64

In [83]:
df.to_csv("datafile.csv", index=False)

In [84]:
#reading the dataset
data = pd.read_csv('/content/datafile.csv')

In [85]:
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [86]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state =0)

In [87]:
#defining and configure the model using KNeighbours
model = KNeighborsClassifier()
# evaluating the model
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.336 (0.034)


Using **Discretization Transform**

In [92]:
#loading dataset
url = "/content/datafile.csv"
dataset = read_csv(url, header=None)

#retrieving the input values
data = dataset.values

#segregating into input and output columns
x, y = data[:, :-1], data[:, -1]
x = x.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))

#defining the pipeline
transf = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
model = KNeighborsClassifier()
pipeline = Pipeline(steps=[('t', transf), ('m', model)])

#evaluating the pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

#performance of the pipeline
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy: 0.310 (0.046)


Using KNN with k-means ordinal discretization transform

In [93]:
#loading dataset
url = "/content/datafile.csv"
dataset = read_csv(url, header=None)

#retrieving the input values
data = dataset.values[:, :-1]

#segregating into input and output columns
x, y = data[:, :-1], data[:, -1]
x = x.astype('float32')
y = LabelEncoder().fit_transform(y.astype('str'))

#defining the pipeline
transf = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
model = KNeighborsClassifier()
pipeline = Pipeline(steps=[('t', transf), ('m', model)])

#evaluating the pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

#performance of the pipeline
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy: 0.997 (0.004)
